In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-03-10 06:32:24,837 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-10 06:32:24,839 - settings - Setting database.user to celiib
INFO - 2021-03-10 06:32:24,840 - settings - Setting database.password to newceliipass
INFO - 2021-03-10 06:32:24,851 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-10 06:32:24,852 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-10 06:32:24,867 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-10 06:32:25,248 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-10 06:32:25,251 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-10 06:32:25,251 - settings - Setting database.user to celiib
INFO - 2021-03-10 06:32:25,252 - settings - Setting database.password to newceliipass
INFO - 2021-03-10 06:32:25,252 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-10 06:32:25,253 - settings - Setting database.user to celiib
INFO - 2021-03-10 06:32:25,254 - settings - Setting database.password to newceliipass
INFO - 2021-03-10 06:32:25,256 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-10 06:32:25,627 - settings - Setting enable_python_native_blobs to True


In [2]:
#so that it will have the adapter defined
from datajoint_utils import *

In [3]:
test_mode = True

# Debugging the contains method

In [4]:
import system_utils as su

In [5]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-10 06:32:27,832 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-10 06:32:27,833 - settings - Setting database.user to celiib
INFO - 2021-03-10 06:32:27,834 - settings - Setting database.password to newceliipass
INFO - 2021-03-10 06:32:27,836 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-10 06:32:27,837 - settings - Setting database.user to celiib
INFO - 2021-03-10 06:32:27,840 - settings - Setting database.password to newceliipass
INFO - 2021-03-10 06:32:27,844 - settings - Setting enable_python_native_blobs to True


Sleeping 185 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-03-10 06:32:28,306 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 9407


In [8]:
# schema.external['decomposition'].delete(delete_external_files=True)

In [9]:
key_source = minnie.Decomposition() & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134988608634,0,0.25,30.00,5,0,1,=BLOB=,1150159,2311814,0,1,1,1,2,11,346,233,11067259.173807988,5580597.123682757,15796.577,104.973,104.973,450.146,379.101,4184,0.00037805204832484123,12.092485549132949,6872596.828,156,0.0006087946237372387,26.821,946998934578.9817,136631721.2458318,85567.61161066596,137793.46559669625,6070505990.890908,4574.8436
864691135065024068,0,0.25,30.00,5,0,1,=BLOB=,1683573,3378174,0,2,2,0,1,11,346,101,15448645.134033982,4735340.661534153,28195.927,309.528,279.925,621.111,612.357,4401,0.00028487935102505666,12.719653179190752,12951735.977,254,0.00033980000888030764,17.327,644008505500.065,97079430.94166055,41687.054101675785,49723.72094703834,2535466557.0868697,7567.1259
864691135155711844,0,0.25,30.00,6,0,1,=BLOB=,1323348,2681606,0,2,2,0,1,12,247,99,10335501.970505765,4157038.1039918233,26480.998,386.808,304.693,553.981,502.772,6481,0.0006270619480790302,26.238866396761132,8914214.834,199,0.0007270410373419097,32.568,1301368737143.5571,126957093.46514548,125912.48503045616,145988.0383609293,6539541392.681191,3068.9409
864691135210232512,0,0.25,30.00,6,0,1,=BLOB=,1104606,2279207,0,5,4,2,2,15,374,97,11682255.066969572,3937393.8053746116,18011.375,128.134,128.134,454.419,431.091,2282,0.00019533899807171054,6.101604278074866,7371490.013,204,0.0003095710631060445,11.186,390388762128.02075,108385813.17371431,33417.24349366473,52959.27437187735,1913670402.588337,2271.6559
864691135341057989,0,0.25,30.00,6,0,1,=BLOB=,910711,1829521,0,3,1,2,2,14,259,112,10163119.200359212,4474894.858952958,28470.197,143.696,143.696,469.631,412.785,3107,0.0003057132302344918,11.996138996138995,6203039.418,137,0.0005008834847936154,22.679,557754542849.0073,106585138.54347737,54880.25200268178,89916.3305702223,4071201042.6934843,2310.7131
864691135345291423,0,0.25,30.00,4,0,1,=BLOB=,708043,1432111,0,0,0,0,1,6,249,195,9952249.61378499,7106458.13114

In [10]:
import numpy as np
import time
import proofreading_utils as pru

split_version = 0
split_version = 1 #fixed the problem with split from suggestions
split_version = 2 #fixed the problem with split from suggestions

verbose = True

@schema
class DecompositionSplit(dj.Computed):
    definition="""
    -> minnie.Decomposition()
    split_index: tinyint unsigned  #the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    split_version: tinyint unsigned  #the version of the splitting algorithm used
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    n_splits             : int unsigned                 # the number of cuts required to help split the neuron
    split_success        : tinyint unsigned             # the successfulness of the splitting
    
    n_error_limbs_cancelled : tinyint unsigned     # number of limbs that couldn't be resolved and cancelled out        
    n_same_soma_limbs_cancelled : tinyint unsigned     # number of same soma touching limbs that couldn't be resolved and cancelled out
    n_multi_soma_limbs_cancelled : tinyint unsigned     # number of multi soma touching limbs that couldn't be resolved and cancelled out        
    
    error_imbs_cancelled_area=NULL : double            # the total area (in microns^2) of the limbs that was cancelled out because touching the same soma multiple times or multiple somas
    error_imbs_cancelled_skeletal_length = NULL : double #the total skeletal length (in microns) of the limbs that were called out because could not be resolved
    
    split_results: longblob #will store the results of how to split the limbs of neuron objects from original neuron
    decomposition: <decomposition>
    
    
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    max_soma_n_faces:  int unsigned                 # The largest number of faces of the somas
    max_soma_volume:  int unsigned                 # The largest volume of the somas the (volume in billions (10*9 nm^3))
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """
                             
    
    ''' Old keysource used for inhibitory excitatory check
    classified_table = (minnie.BaylorManualCellType() &
                        'nucleus_version=3' & 
                        "(cell_type = 'excitatory') or  (cell_type = 'inhibitory')")
    
    key_source = ((minnie.Decomposition & 
                (minnie.NeuronSplitSuggestions.proj()) & 
                (classified_table.proj()) 
                & f"n_somas<{max_n_somas}" & "n_error_limbs>0"))'''
    
    # This keysource acounts that you could have more than 1 possible soma but not a significant limb connecting them (no error limbs)
    key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decomposed neurong object from Decomposition table
        3) Run the multi_soma split suggestions algorithm
        4) Get the number of splits required for this neuron
        5) Split the neuron into a list of neuron objects
        6) For each neuron object in the list:
        - get the number of errored limbs (to indicate the success type)
        - Change the description to include the multiplicity
        - Compute the information on the largest soma faces and volume
        - Save the neuron object to the external
        - Add the new write key to a list to commit 
        7) Write all of the keys 
        """
        
        whole_pass_time = time.time()
        
        
        
        # 1) Get the segment id from the key
        segment_id = key["segment_id"]
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        
        
        # 2) Get the decomposed neuron object from Decomposition table and the split suggestions
#         try:
#             neuron_obj = (minnie.Decomposition & key).fetch1("decomposition")
#         except:
#             neuron_obj = du.fetch_neuron_obj_manual(segment_id)
            
        neuron_obj = du.fetch_neuron_obj_manual(segment_id)
        
        """ Old way that downloaded from another table
        # 3) Retrieve the multi soma suggestions
        split_results = (minnie.NeuronSplitSuggestions & key).fetch1("split_results")
        """
        #3) Calculated the split results
        split_results = pru.multi_soma_split_suggestions(neuron_obj,plot_intermediates=False)
        
        # 4) Get the number of splits required for this neuron
        n_paths_cut = pru.get_n_paths_cut(split_results)

        if verbose:
            print(f"n_paths_cut = {n_paths_cut}")
            
            
        # 5) Split the neuron into a list of neuron objects
        (neuron_list,
        neuron_list_errored_limbs_area,
         neuron_list_errored_limbs_skeletal_length,
        neuron_list_n_multi_soma_errors,
        neuron_list_n_same_soma_errors) = pru.split_neuron(neuron_obj,
                        limb_results=split_results,
                                       verbose=verbose,
                                        return_error_info=True
                                            )
        
        print(f"neuron_list = {neuron_list}")
        print(f"neuron_list_errored_limbs_area = {neuron_list_errored_limbs_area}")
        print(f"neuron_list_n_multi_soma_errors = {neuron_list_n_multi_soma_errors}")
        print(f"neuron_list_n_same_soma_errors = {neuron_list_n_same_soma_errors}")
        
        
        if verbose:
            print(f"Number of neurons: {len(neuron_list)}")
        
        neuron_entries = []
        for neuron_idx in range(len(neuron_list)):
            
            """
            # 6) For each neuron object in the list:
            # - get the number of errored limbs (to indicate the success type)
            # - Compute the information on the largest soma faces and volume
            # - Save the neuron object to the external
            # - Add the new write key to a list to commit 
            """
            n = neuron_list[neuron_idx]
            
            error_imbs_cancelled_area = neuron_list_errored_limbs_area[neuron_idx]
            error_imbs_cancelled_skeletal_length = neuron_list_errored_limbs_skeletal_length[neuron_idx]
            n_multi_soma_limbs_cancelled = neuron_list_n_multi_soma_errors[neuron_idx]
            n_same_soma_limbs_cancelled = neuron_list_n_same_soma_errors[neuron_idx]
            
            
            #for n in neuron_list:
            #     nviz.visualize_neuron(n,
            #                          limb_branch_dict="all")

            # - get the number of errored limbs (to indicate the success type)
            if n.n_error_limbs == 0:
                split_success = 0
            elif n.multi_soma_touching_limbs == 0:
                split_successs = 1
            elif n.same_soma_multi_touching_limbs == 0:
                split_success = 2
            else:
                split_success = 3

            if verbose:
                print(f"split_success = {split_success}")

            # - Compute the information on the largest soma faces and volume
            soma_volumes = [n[k].volume/1000000000 for k in n.get_soma_node_names()] 
            soma_n_faces = [len(n[k].mesh.faces) for k in n.get_soma_node_names()] 

            largest_n_faces = np.max(soma_n_faces)
            largest_volume = np.max(soma_volumes)

            if verbose:
                print(f"largest_n_faces = {largest_n_faces}")
                print(f"largest_volume = {largest_volume}")

            if "split" not in n.description:
                n.description += "_soma_0_split"
                
            #6) Save the file in a certain location
            if True:
                save_time = time.time()
                ret_file_path = n.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                                  return_file_path=True,
                                                 export_mesh=False,
                                                 suppress_output=True)

                ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
                print(f"Save time = {time.time() - save_time}")
            else:
                print("Storing a dummy value for neuron")
                ret_file_path_str = "dummy"



            #7) Pass stats and file location to insert
            new_key = dict(key,
                           split_index = neuron_idx,
                           split_version = split_version,
                           
                           multiplicity=len(neuron_list),

                           n_splits = n_paths_cut,
                           split_success = split_success,
                           
                           n_error_limbs_cancelled = len(error_imbs_cancelled_area),
                           
                           n_multi_soma_limbs_cancelled =n_multi_soma_limbs_cancelled,
                           n_same_soma_limbs_cancelled = n_same_soma_limbs_cancelled,
                           error_imbs_cancelled_area = np.round(np.sum(error_imbs_cancelled_area),4),
                           error_imbs_cancelled_skeletal_length = np.round(np.sum(error_imbs_cancelled_skeletal_length)/1000,4),
                           
                           split_results=split_results,

                           max_soma_n_faces = largest_n_faces,
                           max_soma_volume = largest_volume,


                           decomposition=ret_file_path_str,
                           n_vertices=len(n.mesh.vertices),
                           n_faces=len(n.mesh.faces),
                           run_time=np.round(time.time() - whole_pass_time,4)
                          )

            stats_dict = n.neuron_stats()
            new_key.update(stats_dict)
            
            keys_to_delete = ["axon_length",
            "axon_area"]

            for k_to_delete in keys_to_delete:
                del new_key[k_to_delete]


            neuron_entries.append(new_key)

        
        self.insert(neuron_entries, allow_direct_insert=True, skip_duplicates=True)
        

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - whole_pass_time} ------")

# Running the Populate

In [12]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
(curr_table).delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

In [13]:
import time
import random
pru = reload(pru)
nru = reload(nru)
import neuron
neuron = reload(neuron)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    DecompositionSplit.populate(reserve_jobs=True, suppress_errors=True)
else:
    DecompositionSplit.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionSplit populate = {time.time() - start_time}")

INFO - 2021-03-10 06:33:19,212 - autopopulate - Found 9 keys to populate
INFO - 2021-03-10 06:33:19,232 - connection - Transaction started
INFO - 2021-03-10 06:33:19,234 - autopopulate - Populating: {'segment_id': 864691135373676873, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': Decimal('30.00'), 'process_version': 6, 'index': 0}


Populate Started



---- Working on Neuron 864691135373676873 ----
Decompressing Neuron in minimal output mode...please wait


  0%|          | 0/7 [00:00<?, ?it/s]

n_paths_cut = 2
using precomputed split suggestions


---Working on Splitting Limb 0 with 2 components----


----Working on seperate_graph 0----


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1064950.  764268.  678164.]---------
Starting_edge inside branches_to_conept = [[1064950.  764268.  678164.]
 [1067200.  760150.  679290.]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [18]
printing out current edge:
[[1064950.  764268.  678164.]
 [1067200.  760150.  679290.]]
edge_endpoints_to_process was empty so exiting loop after 32 iterations
starting_node in concept map (that should match the starting edge) = 21
Total time for branches to concept conversion = 1.50673246383667

Done generating concept network 




----Working on seperate_graph 1----


---------Working on soma_idx = 1, soma_group_idx 0, endpt = [1078380.  854492.  695621.]---------
Starting_edge inside branches_to_conept = [[1076150.  842788.  690957.]
 [10783



---------Working on soma_idx = 0, soma_group_idx 0, endpt = [634530. 805769. 852857.]---------
Starting_edge inside branches_to_conept = [[609015. 818596. 806419.]
 [634530. 805769. 852857.]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [426]
printing out current edge:
[[634530. 805769. 852857.]
 [609015. 818596. 806419.]]
edge_endpoints_to_process was empty so exiting loop after 645 iterations
starting_node in concept map (that should match the starting edge) = 403
Total time for branches to concept conversion = 9.50480580329895

Done generating concept network 




----Working on seperate_graph 1----


---------Working on soma_idx = 1, soma_group_idx 0, endpt = [560718.46582278 820460.90949367 900002.82974684]---------
Starting_edge inside branches_to_conept = [[558068.25       821294.75       904011.        ]
 [560718.46582278 820460.90949367 900002.82974684]]
At the start, starting_node (in terms of the skeleton, that shouldn't 

...

  0%|          | 0/12 [00:00<?, ?it/s]

n_paths_cut = 3
using precomputed split suggestions


---Working on Splitting Limb 0 with 3 components----


----Working on seperate_graph 0----


---------Working on soma_idx = 1, soma_group_idx 0, endpt = [695819. 494134. 938560.]---------
Starting_edge inside branches_to_conept = [[695819. 494134. 938560.]
 [696256. 481161. 940207.]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [15]
printing out current edge:
[[695819. 494134. 938560.]
 [696256. 481161. 940207.]]
edge_endpoints_to_process was empty so exiting loop after 27 iterations
starting_node in concept map (that should match the starting edge) = 16
Total time for branches to concept conversion = 0.46569085121154785

Done generating concept network 




----Working on seperate_graph 1----


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [ 631314.  457478. 1015400.]---------
Starting_edge inside branches_to_conept = [[ 631314.  457478. 1015400.]
 [ 632364.  440024.



------ Working on Soma 0 -------
limb_neighbors = [1, 2, 7, 9, 12, 13, 15, 19]
limb_neighbors BEFORE error limbs removed = [1, 2, 7, 9, 12, 13, 15, 19]
limb_neighbors AFTER error limbs removed = [ 1  2  7  9 12 13 15 19]
curr_n_multi_soma_limbs_cancelled = 0
curr_n_same_soma_limbs_cancelled = 0
n_errored_lims = 0
curr_error_limbs_cancelled_area = []
local_floating_meshes = [<trimesh.Trimesh(vertices.shape=(2031, 3), faces.shape=(3450, 3))>]
Already have preprocessed data
--- 1) Finished unpacking preprocessed materials: 8.630752563476562e-05
total_edges = [['S0', 'L0'], ['S0', 'L1'], ['S0', 'L2'], ['S0', 'L3'], ['S0', 'L4'], ['S0', 'L5'], ['S0', 'L6'], ['S0', 'L7']]
--- 2) Finished creating neuron connectivity graph: 0.0006008148193359375
Having to generate soma_meshes_face_idx because none in preprocessed data
--- 3a) Finshed generating soma_meshes_face_idx: 0.2453022003173828
Using precomputed volume ratio
--- 3b) Finished soma creation: 0.09094977378845215
--- 3) Finshed generatin

curr_limb_concept_networks= {0: [<networkx_utils.GraphOrderedEdges object at 0x7f205f359b00>]}
curr_limb_concept_networks= {0: [<networkx_utils.GraphOrderedEdges object at 0x7f205f359550>]}
curr_limb_concept_networks= {0: [<networkx_utils.GraphOrderedEdges object at 0x7f205f359f60>]}
curr_limb_concept_networks= {0: [<networkx_utils.GraphOrderedEdges object at 0x7f205f359ac8>]}
--- 4) Finshed generating Limb objects and adding them to concept graph: 5.200249671936035
--- 5) SKIPPING Doing the adaptive mesh correspondence on the meshparty preprocessing ---
--- 6) SKIPPING Using the computed_attribute_dict to populate neuron attributes ---
self.n_limbs = 8
7) Calculating the spines for the neuorn if do not already exist
Total time for neuron instance creation = 10.754271507263184


------ Working on Soma 1 -------
limb_neighbors = [3, 8, 10, 11, 12, 13, 14, 15, 16, 17]
limb_neighbors BEFORE error limbs removed = [3, 8, 10, 11, 12, 13, 14, 15, 16, 17]
limb_neighbors AFTER error limbs remov



---------Working on soma_idx = 0, soma_group_idx 0, endpt = [ 878623.  650444. 1043320.]---------
Starting_edge inside branches_to_conept = [[ 871202.  607152. 1044710.]
 [ 878623.  650444. 1043320.]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [97]
printing out current edge:
[[ 878623.  650444. 1043320.]
 [ 871202.  607152. 1044710.]]
edge_endpoints_to_process was empty so exiting loop after 423 iterations
starting_node in concept map (that should match the starting edge) = 88
Total time for branches to concept conversion = 7.039511203765869

Done generating concept network 




----Working on seperate_graph 6----
There was no starting information so doing to put dummy information and random starting endpoint = [ 904075.55079855  842363.24915115 1059833.81785287]


---------Working on soma_idx = -1, soma_group_idx -1, endpt = [ 904075.55079855  842363.24915115 1059833.81785287]---------
Starting_edge inside branches_to_conept = [[